**IMPORT LIBRARIES**

In [1]:
import pandas as pd
import openpyxl
import numpy as np

**READING EXCEL**

In [ ]:
init_df=pd.read_excel('data_akbilgic.xlsx',header=1)
init_df.head()

**PREPROCESSING**

In [ ]:
def data_preprocessing(df):
    df=df.loc[0:529] #keep the rows from 0 to 529 (first 530 rows)
    df=df.drop(columns=['date']) #drop date column date
    #df.to_csv('./data_akbilgic_clean.csv', header=True, index=False ,sep=';') #save the cleansed data in a csv file.
    return df
preprocessed_df=data_preprocessing(init_df)
preprocessed_df.head()

**MAKE THE DATA SUPERVISED**

In [ ]:
def timeseries_to_supervised(df, n_in, n_out):
   agg = pd.DataFrame()

   for i in range(n_in, 0, -1):
      df_shifted = df.shift(i).copy()
      df_shifted.rename(columns=lambda x: ('%s(t-%d)' % (x, i)), inplace=True)
      agg = pd.concat([agg, df_shifted], axis=1)

   for i in range(0, n_out):
      df_shifted = df.shift(-i).copy()
      if i == 0:
         df_shifted.rename(columns=lambda x: ('%s(t)' % (x)), inplace=True)
      else:
         df_shifted.rename(columns=lambda x: ('%s(t+%d)' % (x, i)), inplace=True)
      agg = pd.concat([agg, df_shifted], axis=1)
   agg.dropna(inplace=True)
   return agg
supervised_df=timeseries_to_supervised(preprocessed_df,4,1)# timeseries consists of 4 input steps.
supervised_df.head()

**Give [samples, steps, features] shape.**

In [19]:
X, y = supervised_df[[('ISE(t-%d)' % i) for i in range(4, 0, -1)]].values, supervised_df['ISE(t)'].values
print(X.shape, y.shape)
samples = supervised_df.shape[0]
steps = 1
print("Samples",samples)
print("Steps",steps)
features_in = 4
features_out = 1

(526, 4) (526,)
Samples 526
Steps 1


In [21]:
X = np.reshape(X, (samples, steps, features_in))# (Samples,steps,input_features)=(263,1,4)
y = np.reshape(y, (samples, steps, features_out))
print("Data input shape:",X.shape,"Data output shape:", y.shape)

Data input shape: (526, 1, 4) Data output shape: (526, 1, 1)
